In [ ]:
import operator as op
import numpy as np
import sklearn, sklearn.tree

def estimate_gini_impurity(feature_values, threshold, labels, polarity): 
    """Compute the gini impurity for comparing a feature value against a threshold under a given polarity
    feature_values: 1D numpy array, feature_values for samples on one feature dimension
    threshold: float
    labels: 1D numpy array, the label of samples, only +1 and -1. 
    polarity: operator type, only operator.gt or operator.le are allowed
    Examples
    -------------
    >>> feature_values = numpy.array([1,2,3,4,5,6,7,8])
    >>> labels = numpy.array([+1,+1,+1,+1, -1,-1,-1,-1])
    >>> for threshold in range(0,8): 
    ...     estimate_gini_impurity(feature_values, threshold, labels, operator.gt)
    0.5
    0.489795918367347
    0.4444444444444444
    0.31999999999999984
    0.0
    0.0
    0.0
    0.0
    >>> for threshold in range(0,8): 
    ...     estimate_gini_impurity(feature_values, threshold, labels, operator.le)
    1.0
    0.0
    0.0
    0.0
    0.0
    0.31999999999999984
    0.4444444444444445
    0.48979591836734704
    """

    # YOUR CODE HERE
    #Boolean array initialization of size equal to the length of features
    #filtered_features = [False]
    filtered_features = polarity(feature_values, threshold)
    f = feature_values[filtered_features]
    if len(f) == 0:
        gini_impurity = 1
        return gini_impurity
    l = labels[filtered_features]
    class1_members = np.logical_and(f, l == +1)
    class2_members = np.logical_and(f, l == -1)
    len_class1 = sum(class1_members)
    len_class2 = sum(class2_members)
    len_class = len_class1 + len_class2
    pr1 = len_class1/len_class
    pr2 = len_class2/len_class
    gini_impurity = pr1*(1-pr1)+ pr2*(1-pr2)
    

    return gini_impurity

In [ ]:
def estimate_gini_impurity_expectation(feature_values, threshold, labels):
    """Compute the expectation of gini impurity given the feature values on one  feature dimension and a threshold 
    feature_values: 1D numpy array, feature_values for samples on one feature dimension
    threshold: float
    labels: 1D numpy array, the label of samples, only +1 and -1. 
    Examples 
    ---------------
    >>> feature_values = numpy.array([1,2,3,4,5,6,7,8])
    >>> labels = numpy.array([+1,+1,+1,+1, -1,-1,-1,-1])
    >>> for threshold in range(0,9): 
    ...     estimate_gini_impurity_expectation(feature_values, threshold, labels)
    0.5
    0.4285714285714286
    0.3333333333333333
    0.1999999999999999
    0.0
    0.1999999999999999
    0.33333333333333337
    0.42857142857142866
    0.5
    """

    # YOUR CODE HERE
    left_gini  = estimate_gini_impurity(feature_values, threshold, labels, op.le)
    right_gini = estimate_gini_impurity(feature_values, threshold, labels, op.gt)
    filtered_features = op.le(feature_values, threshold)
    number = len(feature_values[filtered_features])
    length = len(feature_values)
    pr1 = number/length

    
    expectation = left_gini * pr1 + right_gini * (1 - pr1)

    return expectation

In [ ]:
def midpoint(x):
    """Given a sequqence of numbers, return the middle points between every two consecutive ones. 
    >>> x= numpy.array([1,2,3,4,5])
    >>> (x[1:] + x[:-1]) / 2
    array([1.5, 2.5, 3.5, 4.5])
    """
    return (x[1:] + x[:-1]) / 2

In [ ]:

def grid_search_split_midpoint(X, y): 
    """Given a dataset, compute the gini impurity expectation for all pairs of features and thresholds. 
    Inputs
    ----------
        X: 2-D numpy array, axis 0 or row is a sample, and axis 1 or column is a feature
        y: 1-D numpy array, the labels, +1 or -1
    Returns
    ---------
        grid: 2-D numpy array, axis 0 or row is a threshold, and axis 1 or column is a feature
    Examples 
    -------------
    >>> numpy.random.seed(1) # fix random number generation starting point
    >>> X = numpy.random.randint(1, 10, (8,3)) # generate training samples
    >>> y = numpy.array([+1,+1,+1,+1, -1,-1,-1,-1])
    >>> grid, feature_id, bts = grid_search_split_midpoint(X, y)
    >>> print (grid)
    [[0.42857143 0.5        0.46666667]
     [0.46666667 0.5        0.46666667]
     [0.46666667 0.46666667 0.46666667]
     [0.375      0.5        0.46666667]
     [0.5        0.5        0.46666667]
     [0.5        0.5        0.5       ]
     [0.5        0.42857143 0.42857143]]
    >>> clf = sklearn.tree.DecisionTreeClassifier(max_depth=1)
    >>> clf = clf.fit(X,y)
    >>> print (clf.tree_.feature[0], clf.tree_.threshold[0], feature_id, bts)
    0 7.0 0 7.0
    >>> print(clf.tree_.feature[0] == feature_id)
    True
    >>> print( clf.tree_.threshold[0] == bts)
    True
    >>> # Antoher test case 
    >>> numpy.random.seed(2) # fix random number generation starting point
    >>> X = numpy.random.randint(1, 30, (8,3)) # generate training samples
    >>> grid, feature_id, bts = grid_search_split_midpoint(X, y)
    >>> print (grid)
    [[0.42857143 0.42857143 0.42857143]
     [0.5        0.5        0.33333333]
     [0.375      0.46666667 0.46666667]
     [0.375      0.5        0.5       ]
     [0.46666667 0.46666667 0.46666667]
     [0.33333333 0.5        0.5       ]
     [0.42857143 0.42857143 0.42857143]]
    >>> clf = clf.fit(X,y) # return the sklearn DT
    >>> print (clf.tree_.feature[0], clf.tree_.threshold[0], feature_id, bts)
    2 8.5 2 8.5
    >>> print(clf.tree_.feature[0] == feature_id)
    True
    >>> print( clf.tree_.threshold[0] == bts)
    True
    >>> # yet antoher test case 
    >>> numpy.random.seed(4) # fix random number generation starting point
    >>> X = numpy.random.randint(1, 100, (8,3)) # generate training samples
    >>> grid, feature_id, bts = grid_search_split_midpoint(X, y)
    >>> print (grid)
    [[0.42857143 0.42857143 0.42857143]
     [0.5        0.5        0.33333333]
     [0.46666667 0.46666667 0.375     ]
     [0.375      0.375      0.375     ]
     [0.46666667 0.2        0.46666667]
     [0.5        0.42857143 0.5       ]
     [0.42857143 0.42857143 0.42857143]]
    >>> clf = clf.fit(X,y) # return the sklearn DT
    >>> print (clf.tree_.feature[0], clf.tree_.threshold[0], feature_id, bts)
    1 47.5 1 47.5
    >>> print(clf.tree_.feature[0] == feature_id)
    True
    >>> print( clf.tree_.threshold[0] == bts)
    True
    """

    X_sorted = np.sort(X, axis=0)
    thresholds = np.apply_along_axis(midpoint, 0, X_sorted)

    # YOUR CODE HERE
    columns = X.shape[1]
    rows    = X.shape[0]-1
    grid = [[0 for i in range(columns)] for j in range(rows)]
    
    for col in range(columns):
        for row in range(rows):
            grid[row][col] = estimate_gini_impurity_expectation(X[:,col], thresholds[row][col], y)
    arr = np.asarray(grid)
    ind = np.unravel_index(np.argmin(arr, axis=None), arr.shape)
    best_threshold = arr[ind]
    best_feature = ind[1]
    

    return grid, best_feature, best_threshold 